## Preprocessing

In [217]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [218]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [219]:
# Determine the number of unique values in each column.
# Code taken from StackOverflow: https://stackoverflow.com/questions/27241253/print-the-unique-values-in-every-column-in-a-pandas-dataframe
application_df.astype("object").describe(include="all").loc["unique", :]

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
Name: unique, dtype: object

In [220]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df["APPLICATION_TYPE"].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [221]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_dict = application_counts.to_dict()
cutoff_value = 1000
application_types_to_replace = [key for key in application_dict if application_dict.get(key) < cutoff_value]

# Replace in dataframe
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app,"Other")

# Check to make sure replacement was successful
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [222]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [223]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# lambda function taken from StackOverflow: https://stackoverflow.com/questions/22320356/pandas-get-values-from-column-that-appear-more-than-x-times
classification_counts.loc[lambda x: x>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

## Optimization: Increased Cutoff Value for "CLASSIFICATION"

In [224]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_dict = classification_counts.to_dict()
cutoff_value = 1000
classifications_to_replace = [key for key in classification_dict if classification_dict.get(key) < cutoff_value]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

## Optimization: Binning "ASK_AMT" Column

In [225]:
# Look at ASK_AMT values to figure out how to bin them
amount_counts = application_df["ASK_AMT"].value_counts()
amount_counts

ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: count, Length: 8747, dtype: int64

In [226]:
# Create arrays for storing cutoff values and labels
amount_dict = amount_counts.to_dict()
cutoff_values = [5000, 10000, 50000, 100000, 500000, 1000000]
cutoff_labels = ["5000", "5001-9999", "10000-49999", "50000-99999", "100000-499999", "500000-999999", ">1000000"]

# Put each amount value in appropriate bin
for index, row in application_df.iterrows():
    if int(row["ASK_AMT"]) == cutoff_values[0]:
        application_df.loc[index, "ASK_AMT"] = cutoff_labels[0]
    elif int(row["ASK_AMT"]) < cutoff_values[1]:
        application_df.loc[index, "ASK_AMT"] = cutoff_labels[1]
    elif int(row["ASK_AMT"]) < cutoff_values[2]:
        application_df.loc[index, "ASK_AMT"] = cutoff_labels[2]
    elif int(row["ASK_AMT"]) < cutoff_values[3]:
        application_df.loc[index, "ASK_AMT"] = cutoff_labels[3]
    elif int(row["ASK_AMT"]) < cutoff_values[4]:
        application_df.loc[index, "ASK_AMT"] = cutoff_labels[4]
    elif int(row["ASK_AMT"]) < cutoff_values[5]:
        application_df.loc[index, "ASK_AMT"] = cutoff_labels[5]
    else:
        application_df.loc[index, "ASK_AMT"] = cutoff_labels[6]

# Check to make sure replacement was successful
application_df["ASK_AMT"].value_counts()

C:\Users\matth\AppData\Local\Temp\ipykernel_9080\2740823329.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '5000' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  application_df.loc[index, "ASK_AMT"] = cutoff_labels[0]


ASK_AMT
5000             25398
10000-49999       2399
100000-499999     2305
>1000000          1577
50000-99999       1421
500000-999999      651
5001-9999          548
Name: count, dtype: int64

In [227]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_copy = application_df.copy().drop(["IS_SUCCESSFUL"], axis=1)
converted_df = pd.concat([pd.get_dummies(application_df_copy[col], dtype=int) for col in application_df_copy.columns], axis=1)
converted_df = pd.concat([converted_df, application_df["IS_SUCCESSFUL"]], axis=1)
converted_df.columns = converted_df.columns.astype(str)
converted_df.head()

,Other,T19,T3,T4,T5,T6,CompanySponsored,Family/Parent,Independent,National,...,N,Y,10000-49999,100000-499999,5000,50000-99999,500000-999999,5001-9999,>1000000,IS_SUCCESSFUL
0,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,1
1,0,0,1,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,1
2,0,0,0,0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,1
4,0,0,1,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,1


In [228]:
# Split our preprocessed data into our features and target arrays
X = converted_df.drop("IS_SUCCESSFUL", axis=1)
y = converted_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [229]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimizations: Increasing Nodes in Hidden Layer and Adding Additional Hidden Layers

In [230]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=47))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

C:\Users\matth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_122 (Dense)               │ (None, 80)             │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_123 (Dense)               │ (None, 50)             │         4,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_124 (Dense)               │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_125 (Dense)               │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_126 (Dense)               │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,041 (50.94 KB)

 Trainable params: 13,041 (50.94 KB)

 Non-trainable params: 0 (0.00 B)

In [231]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [232]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 731us/step - accuracy: 0.7107 - loss: 0.5844
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 726us/step - accuracy: 0.7320 - loss: 0.5525
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 720us/step - accuracy: 0.7379 - loss: 0.5439
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 719us/step - accuracy: 0.7361 - loss: 0.5487
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 724us/step - accuracy: 0.7353 - loss: 0.5485
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 724us/step - accuracy: 0.7359 - loss: 0.5484
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 737us/step - accuracy: 0.7387 - loss: 0.5432
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - accuracy: 0.7370 - loss: 0.5440
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - accuracy: 0.7381 - loss: 0.5418
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 745us/step - accuracy: 0.7358 - loss: 0.5446
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 698us/step - accuracy: 0.7388 - loss: 0.5392
Epoch 12/100
804/80

In [233]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 959us/step - accuracy: 0.7201 - loss: 0.8694
Loss: 0.8694005608558655, Accuracy: 0.7201166152954102


In [234]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")